In [42]:
import pandas as pd
import os

In [44]:
d = pd.read_csv("data/twtc.csv")
scouting_data = d[['text', 'label']]
print(scouting_data.shape)
scouting_data.head()

(9175, 2)


,text,label
0,PERSON is a Level 1 sex offender and wouldn't ...,0
1,The way teams value PERSON depends on whether ...,0
2,PERSON PERSON was the headliner in the three-p...,-1
3,PERSON made headlines for all the wrong reason...,-1
4,"The ORGANIZATION have acquired PERSON twice, ...",1


In [45]:
sports_files = os.listdir("data/SportsArticles/Raw data")
texts = []
for filename in sports_files:
    path = "data/SportsArticles/Raw data/" + filename
    with open(path, encoding = "utf8", errors = 'ignore') as f:
        t = f.read()
        texts.append(t)
sports_article_data = pd.DataFrame(texts, columns = ['text'])
print(sports_article_data.shape)
sports_article_data.head()

(1000, 1)


,text
0,With a world title and an Olympic silver medal...
1,The Croatia striker took advantage of a mistak...
2,Round 1 – Herman comes out with a low kick and...
3,Zagoruiko skied fast and shot clean to take a ...
4,Emily Seebohm blamed her overuse of social net...


In [46]:
wiki_data = pd.read_csv("data/wiki_sentences.txt", sep = '\n', header = None, on_bad_lines = 'skip', names = ['text'])
print(wiki_data.shape)
wiki_data.head()

(7867181, 1)


,text
0,"0.000123, which corresponds to a distance of 7..."
1,"000webhost is a free web hosting service, oper..."
2,"0010x0010 is a Dutch-born audiovisual artist, ..."
3,0-0-1-3 is an alcohol abuse prevention program...
4,"0.01 is the debut studio album of H3llb3nt, re..."
